# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119044e+02     1.459687e+00
 * time: 0.09062504768371582
     1     1.205559e+01     8.424775e-01
 * time: 0.9645121097564697
     2    -1.170489e+01     9.997701e-01
 * time: 1.1135361194610596
     3    -3.378313e+01     7.342869e-01
 * time: 1.2837719917297363
     4    -4.730336e+01     5.245068e-01
 * time: 1.4393179416656494
     5    -5.675638e+01     1.870099e-01
 * time: 1.5867400169372559
     6    -5.970681e+01     1.054880e-01
 * time: 1.695322036743164
     7    -6.081030e+01     5.441595e-02
 * time: 1.8111090660095215
     8    -6.120922e+01     6.994394e-02
 * time: 1.9271371364593506
     9    -6.154066e+01     3.652880e-02
 * time: 2.0351030826568604
    10    -6.176374e+01     2.564564e-02
 * time: 2.1517720222473145
    11    -6.194173e+01     2.274283e-02
 * time: 2.2597529888153076
    12    -6.200818e+01     2.163160e-02
 * time: 2.3743810653686523
    13    -6.208111e+01     1.648755e-02
 * time: 2.4885540008

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671080
    AtomicLocal         -18.8557674
    AtomicNonlocal      14.8522625
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485382 
    Xc                  -19.3336825

    total               -62.261666455686
